In [4]:
!pip install pytorch-crf

In [5]:
import math
import os
import json
import re
import random

import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.utils.rnn as rnn_utils

from torch.nn.utils.rnn import pad_sequence
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split

from collections import defaultdict

from torchcrf import CRF

from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [6]:
train = pd.read_csv('/content/drive/My Drive/Datasets/Coleridge/datasets/train.csv')
train_items = train.sample(n=500, random_state=42)

X_train, X_test = train_test_split(train_items, test_size=0.1, random_state=42)
train_papers = {}
test_papers = {}

for i in range(len(X_train)):
    curr_path = os.path.join(
        os.getcwd(),
        'drive',
        'My Drive',
        'Datasets',
        'Coleridge',
        'datasets',
        'train',
        X_train.iloc[i]['Id'] + '.json')
    with open(curr_path, 'r') as file:
        curr_json = json.load(file)
        train_papers[X_train.iloc[i]['Id']] = curr_json

for i in range(len(X_test)):
    curr_path = os.path.join(
        os.getcwd(),
        'drive',
        'My Drive',
        'Datasets',
        'Coleridge',
        'datasets',
        'train',
        X_test.iloc[i]['Id'] + '.json')
    with open(curr_path, 'r') as file:
        curr_json = json.load(file)
        test_papers[X_test.iloc[i]['Id']] = curr_json

In [7]:
# Hyperparameters
MAX_LENGTH = 64 # max no. words for each sentence.
OVERLAP = 20 # if a sentence exceeds MAX_LENGTH, we split it to multiple sentences with overlapping

MAX_SAMPLE = None # set a small number for experimentation, set None for production.


def clean_training_text(txt):
    """
    similar to the default clean_text function but without lowercasing.
    """
    return re.sub('[^A-Za-z0-9]+', ' ', str(txt)).strip()

def shorten_sentences(sentences):
    short_sentences = []
    for sentence in sentences:
        words = sentence.split()
        if len(words) > MAX_LENGTH:
            for p in range(0, len(words), MAX_LENGTH - OVERLAP):
                short_sentences.append(' '.join(words[p:p+MAX_LENGTH]))
        else:
            short_sentences.append(sentence)
    return short_sentences

def find_sublist(big_list, small_list):
    all_positions = []
    for i in range(len(big_list) - len(small_list) + 1):
        if small_list == big_list[i:i+len(small_list)]:
            all_positions.append(i)

    return all_positions

def tag_sentence(sentence, labels): # requirement: both sentence and labels are already cleaned
    sentence_words = sentence.split()

    if labels is not None and any(re.findall(f'\\b{label}\\b', sentence)
                                  for label in labels): # positive sample
        nes = ['O'] * len(sentence_words)
        for label in labels:
            label_words = label.split()

            all_pos = find_sublist(sentence_words, label_words)
            for pos in all_pos:
                nes[pos] = 'B'
                for i in range(pos+1, pos+len(label_words)):
                    nes[i] = 'I'

        return True, list(zip(sentence_words, nes))

    else: # negative sample
        nes = ['O'] * len(sentence_words)
        return False, list(zip(sentence_words, nes))

In [8]:
import re
from tqdm import tqdm
import random

cnt_pos, cnt_neg = 0, 0 # number of sentences that contain/not contain labels
ner_data = []

pbar = tqdm(total=len(train_items))
for i, id, dataset_label in X_train[['Id', 'dataset_label']].itertuples():
    # paper
    paper = train_papers[id]

    # labels
    labels = dataset_label.split('|')
    labels = [clean_training_text(label) for label in labels]

    # sentences
    sentences = set([clean_training_text(sentence) for section in paper
                 for sentence in section['text'].split('.')
                ])
    sentences = shorten_sentences(sentences) # make sentences short
    sentences = [sentence for sentence in sentences if len(sentence) > 10] # only accept sentences with length > 10 chars

    # positive sample
    for sentence in sentences:
        is_positive, tags = tag_sentence(sentence, labels)
        if is_positive:
            cnt_pos += 1
            ner_data.append(tags)
        elif any(word in sentence.lower() for word in ['data', 'study']):
            ner_data.append(tags)
            cnt_neg += 1

    # process bar
    pbar.update(1)
    pbar.set_description(f"Training data size: {cnt_pos} positives + {cnt_neg} negatives")

# shuffling
random.shuffle(ner_data)

Training data size: 1153 positives + 18588 negatives:  90%|█████████ | 450/500 [00:09<00:00, 55.36it/s]

In [9]:
print(len(sentences[2]))
print(sentences[2])

262
It has been noted that the relative importance of different slices in a sMRI scan can be very diverse which indicates that the diversity in the level of information in individual slices of a sMRI scan can also have an impact on the overall accuracy of the system


In [10]:
def build_mappings(data):
    word_set = set()
    tag_set = set()

    # Iterate through the dataset to collect all unique words and tags
    for sentence in data:
        for word, tag in sentence:
            word_set.add(word)
            tag_set.add(tag)

    # Create word2idx mapping, starting with special tokens
    word2idx = {'<PAD>': 0, '<UNK>': 1}
    for idx, word in enumerate(word_set, start=2):  # Start indexing from 2 to leave 0 and 1 for special tokens
        word2idx[word] = idx

    # Create tag2idx mapping
    tag2idx = {'O': 0}  # Often 'O' is the default label, so start indexing from 0
    for idx, tag in enumerate(tag_set):  # Start from 1 because 0 is reserved for 'O'
        if tag != 'O':  # Ensure 'O' is always mapped to 0
            tag2idx[tag] = idx

    return word2idx, tag2idx


word2idx, tag2idx = build_mappings(ner_data)
print(word2idx)
print(tag2idx)

{'<PAD>': 0, '<UNK>': 1, 'Gbps': 2, 'infestation': 3, 'randomized': 4, 'NCAR': 5, 'Importantly': 6, 'authoritarian': 7, 'spheroid': 8, 'percentile': 9, 'prompting': 10, '51k': 11, 'Sadler': 12, 'Semantic': 13, 'According': 14, 'NSoFaS': 15, '605': 16, 'Hurricanes': 17, 'restructuring': 18, 'configuration': 19, 'Gmelin': 20, 'tests': 21, 'uploads': 22, 'Geographic': 23, 'circumstances': 24, 'Atlantic': 25, 'Partnership': 26, '850': 27, 'thresholding': 28, 'Aljabar': 29, 'cows': 30, 'reentry': 31, 'Beam': 32, 'Assuming': 33, 'protected': 34, 'outdated': 35, 'offshore': 36, 'pertain': 37, 'IDIBAPS': 38, '5652': 39, 'Attracting': 40, 'Wales': 41, 'Estonia': 42, 'Woodworth': 43, 'Woodpecker': 44, 'Hendricks': 45, 'suspiciously': 46, 'F2TRP2WT': 47, 'FIT': 48, 'bilateral': 49, 'figures': 50, 'Fowler': 51, 'burdening': 52, 'researchers': 53, '1860': 54, '004': 55, 'Represen': 56, 'advice': 57, 'stems': 58, 'affiliate': 59, 'knots': 60, 'rotated': 61, 'equivalently': 62, 'process': 63, 'optimu

In [11]:
positive_samples = [sample for sample in ner_data if any(tag in ['B', 'I'] for _, tag in sample)]
negative_samples = [sample for sample in ner_data if all(tag == 'O' for _, tag in sample)]

In [12]:
class NERDataset(Dataset):
    def __init__(self, data, word2idx, tag2idx):
        """
        Args:
            data (list of list of tuples): The dataset where each element is a list of (word, tag) tuples.
            word2idx (dict): Mapping from words to indices.
            tag2idx (dict): Mapping from tags to indices.
            max_len (int): Maximum length for padding.
        """
        self.data = data
        self.word2idx = word2idx
        self.tag2idx = tag2idx

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sentence = self.data[idx]

        # Separate words and tags from the tuples
        words = [w for w, t in sentence]
        tags = [t for w, t in sentence]

        # Convert words and tags to their corresponding indices
        word_ids = [self.word2idx.get(w, self.word2idx['<UNK>']) for w in words]
        tag_ids = [self.tag2idx.get(t, self.tag2idx['O']) for t in tags]

        # Convert to tensors
        return torch.tensor(word_ids), torch.tensor(tag_ids)

def collate_fn(batch):
    words, tags = zip(*batch)

    words_padded = pad_sequence(words, batch_first=True, padding_value=word2idx['<PAD>'])
    tags_padded = pad_sequence(tags, batch_first=True, padding_value=tag2idx['O'])

    return words_padded, tags_padded

In [13]:
batch_size = 32
batches = []

# Loop through positive samples and add balanced batches
for i in range(0, len(positive_samples), batch_size // 2):
    pos_batch = positive_samples[i:i + batch_size // 2]
    neg_batch = negative_samples[i:i + batch_size // 2]
    batch = pos_batch + neg_batch
    random.shuffle(batch)
    batches.append(batch)


dataloaders = []
for batch in batches:
    dataset = NERDataset(batch, word2idx, tag2idx)
    dataloader = DataLoader(dataset, batch_size=batch_size, collate_fn=collate_fn)
    dataloaders.append(dataloader)

In [14]:
class NERLSTM(nn.Module):
    def __init__(self, vocab_size, tagset_size, embedding_dim=128, hidden_dim=128):
        super(NERLSTM, self).__init__()

        self.embedding_layer = nn.Embedding(vocab_size, embedding_dim, padding_idx=word2idx['<PAD>'])

        # LSTM
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)

        self.fc = nn.Linear(hidden_dim, tagset_size)

    def forward(self, x):
        # x: (batch_size, max_len)
        embeds = self.embedding_layer(x)
        # print('embeds: ', embeds)

        lstm_out, _  = self.lstm(embeds)
        # print('lstm out: ', lstm_out)

        tag_scores = self.fc(lstm_out)
        # print('tag scores', tag_scores)

        return tag_scores


def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            m.bias.data.fill_(0.01)
    elif isinstance(m, nn.LSTM):
        for name, param in m.named_parameters():
            if 'weight' in name:
                nn.init.xavier_uniform_(param)
            elif 'bias' in name:
                param.data.fill_(0)

In [15]:
import torch
import math

embedding_dim = 128
hidden_dim = 32
vocab_size = len(word2idx)
tagset_size = len(tag2idx)
max_len = 50

model = NERLSTM(vocab_size, tagset_size, embedding_dim, hidden_dim)
model.apply(init_weights)
# model = SimpleNERModel(vocab_size, tagset_size, embedding_dim)

class_weights = torch.tensor([1.0, 10.0, 10.0])  # Example weights for 'O', 'B', 'I'
loss_function = nn.CrossEntropyLoss(weight=class_weights, ignore_index=word2idx['<PAD>'])
optimizer = torch.optim.Adam(model.parameters(), lr=1e-6)

dataset = NERDataset(ner_data, word2idx, tag2idx)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True, collate_fn=collate_fn)

epochs = 10
for epoch in range(epochs):
    total_loss = 0.0
    for batch in dataloader:
        words, tags = batch

        if torch.all(words == word2idx['<PAD>']):
            print("All words in this batch are padding!")
            continue  # Skip this batch

        if torch.isnan(words).any():
            print("NaN detected in input!")

        optimizer.zero_grad()

        outputs = model(words)

        outputs = outputs.view(-1, tagset_size)
        tags = tags.view(-1)

        loss = loss_function(outputs, tags)

        # Check for NaNs in loss
        if math.isnan(loss.item()):
            # print(words.values(), tags.values())
            print("NaN loss encountered, exiting.")
            break

        loss.backward()

        for name, param in model.named_parameters():
            if param.grad is not None and torch.isnan(param.grad).any():
                print(f"NaN gradient found in {name}")

        # Clip gradients to avoid exploding gradients
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.5)

        optimizer.step()

        loss_item = loss.item()

        # Check for NaNs in model parameters
        for name, param in model.named_parameters():
            if torch.isnan(param.grad).any():
                print(f"NaN in gradients of {name}")

        total_loss += loss_item

    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(dataloader)}")


NaN loss encountered, exiting.
Epoch 1/10, Loss: 0.19271609949062557


Training data size: 1153 positives + 18588 negatives:  90%|█████████ | 450/500 [00:20<00:00, 55.36it/s]

NaN loss encountered, exiting.
Epoch 2/10, Loss: 0.3214148151835963
NaN loss encountered, exiting.
Epoch 3/10, Loss: 0.051200576584701785
NaN loss encountered, exiting.
Epoch 4/10, Loss: 0.0
NaN loss encountered, exiting.
Epoch 5/10, Loss: 0.037886692482290915
NaN loss encountered, exiting.
Epoch 6/10, Loss: 0.23911272785038623
NaN loss encountered, exiting.
Epoch 7/10, Loss: 0.11311328758313818
NaN loss encountered, exiting.
Epoch 8/10, Loss: 0.6090982535124597
NaN loss encountered, exiting.
Epoch 9/10, Loss: 0.12137716873563995
NaN loss encountered, exiting.
Epoch 10/10, Loss: 0.11830369824344672


In [16]:
cnt_pos_test, cnt_neg_test = 0, 0  # number of sentences that contain/not contain labels
ner_data_test = []

pbar = tqdm(total=len(X_test))
for i, id, dataset_label in X_test[['Id', 'dataset_label']].itertuples():
    # paper
    paper = test_papers[id]

    # labels
    labels = dataset_label.split('|')
    labels = [clean_training_text(label) for label in labels]

    # sentences
    sentences = set([clean_training_text(sentence) for section in paper
                 for sentence in section['text'].split('.')
                ])
    sentences = shorten_sentences(sentences)  # make sentences short
    sentences = [sentence for sentence in sentences if len(sentence) > 10]  # only accept sentences with length > 10 chars

    # positive sample
    for sentence in sentences:
        is_positive, tags = tag_sentence(sentence, labels)
        if is_positive:
            cnt_pos_test += 1
            ner_data_test.append(tags)
        elif any(word in sentence.lower() for word in ['data', 'study']):
            ner_data_test.append(tags)
            cnt_neg_test += 1

    # process bar
    pbar.update(1)
    pbar.set_description(f"Test data size: {cnt_pos_test} positives + {cnt_neg_test} negatives")

# Shuffle the test data
random.shuffle(ner_data_test)


Test data size: 1 positives + 50 negatives:   2%|▏         | 1/50 [00:00<00:00, 67.88it/s]
Training data size: 1153 positives + 18588 negatives:  90%|█████████ | 450/500 [00:59<00:06,  7.54it/s]

Test data size: 93 positives + 2049 negatives: 100%|██████████| 50/50 [00:00<00:00, 55.90it/s]

In [17]:
# Create the test dataset and dataloader
test_dataset = NERDataset(ner_data_test, word2idx, tag2idx)
test_dataloader = DataLoader(test_dataset, batch_size=2, shuffle=False, collate_fn=collate_fn)

In [18]:
from sklearn.metrics import classification_report

# Assuming test set is loaded similarly as the training set
model.eval()
true_tags, pred_tags = [], []
with torch.no_grad():
    for batch in test_dataloader:
        words, tags = batch
        outputs = model(words)
        predictions = torch.argmax(outputs, dim=2)  # (batch_size, max_len)

        true_tags.extend(tags.view(-1).tolist())
        pred_tags.extend(predictions.view(-1).tolist())

# Filter out padding tokens and calculate metrics
true_tags = [tag for tag in true_tags if tag != word2idx['<PAD>']]
pred_tags = [tag for tag in pred_tags if tag != word2idx['<PAD>']]


In [19]:
from sklearn.metrics import classification_report

def evaluate_model(model, dataloader, word2idx, tag2idx):
    model.eval()  # Set the model to evaluation mode
    all_true_tags = []
    all_pred_tags = []

    with torch.no_grad():  # Disable gradient calculation for evaluation
        for batch in dataloader:
            words, true_tags = batch

            # Get the model predictions
            outputs = model(words)
            predictions = torch.argmax(outputs, dim=-1)  # Get the tag with the highest score

            # Convert the predictions and true tags to lists
            true_tags = true_tags.view(-1).cpu().numpy()
            predictions = predictions.view(-1).cpu().numpy()

            # Mask out padding tokens (ignore_index for the '<PAD>' token)
            mask = (words.view(-1) != word2idx['<PAD>']).cpu().numpy()

            true_tags = true_tags[mask]
            predictions = predictions[mask]

            # Collect all true and predicted tags
            all_true_tags.extend(true_tags)
            all_pred_tags.extend(predictions)

    # Print classification report
    print(classification_report(all_true_tags, all_pred_tags, target_names=[key for key in tag2idx.keys()]))

evaluate_model(model, test_dataloader, word2idx, tag2idx)

              precision    recall  f1-score   support

           O       1.00      0.43      0.60     54074
           B       0.00      0.14      0.00       102
           I       0.00      0.36      0.01       192

    accuracy                           0.43     54368
   macro avg       0.33      0.31      0.20     54368
weighted avg       0.99      0.43      0.60     54368



# CRFModel

In [20]:
# Custom collate function for padding sequences
def collate_fn(batch):
    # Separate words and tags from the batch
    words, tags = zip(*batch)

    # Pad sequences of words and tags
    padded_words = pad_sequence(words, batch_first=True, padding_value=word2idx['<PAD>'])
    padded_tags = pad_sequence(tags, batch_first=True, padding_value=tag2idx['O'])  # 'O' is typically the padding tag

    return padded_words, padded_tags

In [21]:
def collate_fn(batch):
    # Separate words and tags from the batch
    words, tags = zip(*batch)

    # Pad sequences of words and tags
    padded_words = pad_sequence(words, batch_first=True, padding_value=word2idx['<PAD>'])
    padded_tags = pad_sequence(tags, batch_first=True, padding_value=tag2idx['O'])  # 'O' is typically the padding tag

    return padded_words, padded_tags


# Sample Dataset class to return features (embeddings) and labels
class CRFDataset(Dataset):
    def __init__(self, data, word2idx, tag2idx, max_len=50):
        self.data = data
        self.word2idx = word2idx
        self.tag2idx = tag2idx
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sentence = self.data[idx]

        # Extract words and tags
        words = [w for w, t in sentence]
        tags = [t for w, t in sentence]

        # Convert words and tags to their corresponding indices
        word_ids = [self.word2idx.get(w, self.word2idx['<UNK>']) for w in words]
        tag_ids = [self.tag2idx.get(t, self.tag2idx['O']) for t in tags]

        # Pad sequences to the max length
        word_ids = word_ids + [self.word2idx['<PAD>']] * (self.max_len - len(word_ids))
        tag_ids = tag_ids + [self.tag2idx['O']] * (self.max_len - len(tag_ids))

        return torch.tensor(word_ids), torch.tensor(tag_ids)


train_dataset = CRFDataset(ner_data, word2idx, tag2idx, max_len=50)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)
test_dataset = CRFDataset(ner_data_test, word2idx, tag2idx, max_len=50)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)

In [22]:
class CRFModel(nn.Module):
    def __init__(self, vocab_size, tagset_size, embedding_dim=128):
        super(CRFModel, self).__init__()
        # Word embedding layer
        self.embedding_layer = nn.Embedding(vocab_size, embedding_dim, padding_idx=word2idx['<PAD>'])

        # Fully connected layer to project embeddings to tag space
        self.fc = nn.Linear(embedding_dim, tagset_size)

        # CRF layer
        self.crf = CRF(tagset_size, batch_first=True)

    def forward(self, x):
        embeds = self.embedding_layer(x)  # Convert tokens to embeddings
        emissions = self.fc(embeds)       # Project embeddings to tag scores
        return emissions  # Return emissions for each token in the sequence

    def loss(self, emissions, tags, mask):
        return -self.crf(emissions, tags, mask=mask)  # Maximize log-likelihood

    def predict(self, emissions, mask):
        return self.crf.decode(emissions, mask=mask)


In [23]:
# Custom function to generate a mask for valid tokens
def generate_mask(words, word2idx):
    return (words != word2idx['<PAD>']).bool()

# Training loop for the standalone CRF model
def train_crf(model, dataloader, optimizer, word2idx, tag2idx, epochs=10):
    for epoch in range(epochs):
        total_loss = 0.0
        model.train()

        for batch in dataloader:
            words, tags = batch
            mask = generate_mask(words, word2idx)

            # Forward pass
            emissions = model(words)

            # Compute loss
            loss = model.loss(emissions, tags, mask)

            # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss / len(dataloader)}")

# Example of how to instantiate the model and optimizer
model = CRFModel(vocab_size=len(word2idx), tagset_size=len(tag2idx))
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Train the CRF model
train_crf(model, train_dataloader, optimizer, word2idx, tag2idx, epochs=10)



Test data size: 93 positives + 2049 negatives: 100%|██████████| 50/50 [00:20<00:00, 55.90it/s]

Epoch 1/10, Loss: 247.85179911015294
Epoch 2/10, Loss: 38.324379896228855
Epoch 3/10, Loss: 19.7207186821021
Epoch 4/10, Loss: 13.368976656389778
Epoch 5/10, Loss: 10.17644743247009
Epoch 6/10, Loss: 8.170482910239716
Epoch 7/10, Loss: 6.8184002438858995
Epoch 8/10, Loss: 5.7743854599988245
Epoch 9/10, Loss: 5.021957884344908
Epoch 10/10, Loss: 4.382945348223372


In [24]:
from sklearn.metrics import classification_report

def evaluate_crf(model, dataloader, word2idx, tag2idx):
    model.eval()  # Set the model to evaluation mode
    all_true_tags = []
    all_pred_tags = []

    with torch.no_grad():  # Disable gradient calculation for evaluation
        for batch in dataloader:
            words, true_tags = batch

            mask = generate_mask(words, word2idx)
            emissions = model(words)
            predictions = model.predict(emissions, mask)

            # Flatten the predictions and true tags
            for i in range(len(predictions)):
                true = true_tags[i][mask[i] == 1].cpu().numpy()
                pred = predictions[i]
                all_true_tags.extend(true)
                all_pred_tags.extend(pred)

    # Get the unique classes present in both true and predicted labels
    unique_labels = sorted(set(all_true_tags) | set(all_pred_tags))

    # Map tag indices to their names
    label_names = [list(tag2idx.keys())[list(tag2idx.values()).index(label)] for label in unique_labels]

    # Print classification report
    print(classification_report(all_true_tags, all_pred_tags, labels=unique_labels, target_names=label_names))

# Evaluate the standalone CRF model on the test set
evaluate_crf(model, test_dataloader, word2idx, tag2idx)


              precision    recall  f1-score   support

           O       1.00      1.00      1.00     54074
           B       0.69      0.71      0.70       102
           I       0.81      0.44      0.57       192

    accuracy                           1.00     54368
   macro avg       0.83      0.72      0.76     54368
weighted avg       1.00      1.00      1.00     54368



# Custom LSTM Cell

In [28]:
class CustomLinear(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(CustomLinear, self).__init__()

        self.W = nn.Parameter(torch.randn(output_dim, input_dim))
        self.b = nn.Parameter(torch.randn(output_dim))

    def forward(self, x):
        return torch.matmul(x, self.W.T) + self.b


class CustomLSTMCell(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(CustomLSTMCell, self).__init__()

        self.hidden_dim = hidden_dim

        self.W_f = CustomLinear(input_dim + hidden_dim, hidden_dim)
        self.b_f = nn.Parameter(torch.zeros(hidden_dim))

        self.W_i = CustomLinear(input_dim + hidden_dim, hidden_dim)
        self.b_i = nn.Parameter(torch.zeros(hidden_dim))

        self.W_C = CustomLinear(input_dim + hidden_dim, hidden_dim)
        self.b_C = nn.Parameter(torch.zeros(hidden_dim))

        self.W_o = CustomLinear(input_dim + hidden_dim, hidden_dim)
        self.b_o = nn.Parameter(torch.zeros(hidden_dim))

    def forward(self, x, h_prev, C_prev):
        combined = torch.cat((x, h_prev), dim=1) # Concatenate input and prev hidden state

        f_t = torch.sigmoid(self.W_f(combined) + self.b_f) # Forget gate

        i_t = torch.sigmoid(self.W_i(combined) + self.b_f) # Input gate

        C_tilde = torch.tanh(self.W_C(combined) + self.b_C) # Candidate cell state

        C_t = f_t * C_prev + i_t * C_tilde # Cell state update

        o_t = torch.sigmoid(self.W_o(combined) + self.b_o) # Output gate

        h_t = o_t * torch.tanh(C_t) # Hidden state update

        return h_t, C_t


class CustomLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(CustomLSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.lstm_cell = CustomLSTMCell(input_dim, hidden_dim)

    def forward(self, x):
        batch_size, seq_len, _ = x.size()

        h_t = torch.zeros(batch_size, self.hidden_dim).to(x.device)
        C_t = torch.zeros(batch_size, self.hidden_dim).to(x.device)

        outputs = []
        for t in range(seq_len):
            h_t, C_t = self.lstm_cell(x[:, t, :], h_t, C_t)
            outputs.append(h_t.unsqueeze(1))

        return torch.cat(outputs, dim=1)

In [29]:
class CustomNERLSTM(nn.Module):
    def __init__(self, vocab_size, tagset_size, embedding_dim=128, hidden_dim=128):
        super(CustomNERLSTM, self).__init__()

        self.embedding_layer = nn.Embedding(vocab_size, embedding_dim, padding_idx=word2idx['<PAD>'])

        self.lstm = CustomLSTM(embedding_dim, hidden_dim)

        self.fc = nn.Linear(hidden_dim, tagset_size)

    def forward(self, x):
        embeds = self.embedding_layer(x)

        lstm_out = self.lstm(embeds)

        tag_scores = self.fc(lstm_out)

        return tag_scores

In [37]:
import torch
import math

embedding_dim = 128
hidden_dim = 32
vocab_size = len(word2idx)
tagset_size = len(tag2idx)
max_len = 50

model = CustomNERLSTM(vocab_size, tagset_size, embedding_dim, hidden_dim)
model.apply(init_weights)
# model = SimpleNERModel(vocab_size, tagset_size, embedding_dim)

class_weights = torch.tensor([1.0, 10.0, 10.0])  # Example weights for 'O', 'B', 'I'
loss_function = nn.CrossEntropyLoss(weight=class_weights, ignore_index=word2idx['<PAD>'])
optimizer = torch.optim.Adam(model.parameters(), lr=1e-6)

dataset = NERDataset(ner_data, word2idx, tag2idx)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True, collate_fn=collate_fn)

epochs = 10
for epoch in range(epochs):
    total_loss = 0.0
    for batch in dataloader:
        words, tags = batch

        if torch.all(words == word2idx['<PAD>']):
            print("All words in this batch are padding!")
            continue  # Skip this batch

        if torch.isnan(words).any():
            print("NaN detected in input!")

        optimizer.zero_grad()

        outputs = model(words)

        outputs = outputs.view(-1, tagset_size)
        tags = tags.view(-1)

        loss = loss_function(outputs, tags)

        # Check for NaNs in loss
        if math.isnan(loss.item()):
            # print(words.values(), tags.values())
            print("NaN loss encountered, exiting.")
            break

        loss.backward()

        for name, param in model.named_parameters():
            if param.grad is not None and torch.isnan(param.grad).any():
                print(f"NaN gradient found in {name}")

        # Clip gradients to avoid exploding gradients
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.5)

        optimizer.step()

        loss_item = loss.item()

        # Check for NaNs in model parameters
        for name, param in model.named_parameters():
            if param.grad is None:
                continue
            if torch.isnan(param.grad).any():
                print(f"NaN in gradients of {name}")

        total_loss += loss_item

    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(dataloader)}")


NaN loss encountered, exiting.
Epoch 1/10, Loss: 0.21517446531061216
NaN loss encountered, exiting.
Epoch 2/10, Loss: 0.04051248097496897
NaN loss encountered, exiting.
Epoch 3/10, Loss: 0.11449269569421663
NaN loss encountered, exiting.
Epoch 4/10, Loss: 0.04752047374410537
NaN loss encountered, exiting.
Epoch 5/10, Loss: 0.00868138217617393
NaN loss encountered, exiting.
Epoch 6/10, Loss: 0.14756610054028457
NaN loss encountered, exiting.
Epoch 7/10, Loss: 0.09423966643108134
NaN loss encountered, exiting.
Epoch 8/10, Loss: 0.01946690668951732
NaN loss encountered, exiting.
Epoch 9/10, Loss: 0.4560527188106648
NaN loss encountered, exiting.
Epoch 10/10, Loss: 0.24093549729936717


In [38]:
cnt_pos_test, cnt_neg_test = 0, 0  # number of sentences that contain/not contain labels
ner_data_test = []

pbar = tqdm(total=len(X_test))
for i, id, dataset_label in X_test[['Id', 'dataset_label']].itertuples():
    # paper
    paper = test_papers[id]

    # labels
    labels = dataset_label.split('|')
    labels = [clean_training_text(label) for label in labels]

    # sentences
    sentences = set([clean_training_text(sentence) for section in paper
                 for sentence in section['text'].split('.')
                ])
    sentences = shorten_sentences(sentences)  # make sentences short
    sentences = [sentence for sentence in sentences if len(sentence) > 10]  # only accept sentences with length > 10 chars

    # positive sample
    for sentence in sentences:
        is_positive, tags = tag_sentence(sentence, labels)
        if is_positive:
            cnt_pos_test += 1
            ner_data_test.append(tags)
        elif any(word in sentence.lower() for word in ['data', 'study']):
            ner_data_test.append(tags)
            cnt_neg_test += 1

    # process bar
    pbar.update(1)
    pbar.set_description(f"Test data size: {cnt_pos_test} positives + {cnt_neg_test} negatives")

# Shuffle the test data
random.shuffle(ner_data_test)

Test data size: 93 positives + 2049 negatives: 100%|██████████| 50/50 [00:00<00:00, 62.54it/s]

In [39]:
# Create the test dataset and dataloader
test_dataset = NERDataset(ner_data_test, word2idx, tag2idx)
test_dataloader = DataLoader(test_dataset, batch_size=2, shuffle=False, collate_fn=collate_fn)

In [40]:
from sklearn.metrics import classification_report

# Assuming test set is loaded similarly as the training set
model.eval()
true_tags, pred_tags = [], []
with torch.no_grad():
    for batch in test_dataloader:
        words, tags = batch
        outputs = model(words)
        predictions = torch.argmax(outputs, dim=2)  # (batch_size, max_len)

        true_tags.extend(tags.view(-1).tolist())
        pred_tags.extend(predictions.view(-1).tolist())

# Filter out padding tokens and calculate metrics
true_tags = [tag for tag in true_tags if tag != word2idx['<PAD>']]
pred_tags = [tag for tag in pred_tags if tag != word2idx['<PAD>']]


In [41]:
from sklearn.metrics import classification_report

def evaluate_model(model, dataloader, word2idx, tag2idx):
    model.eval()  # Set the model to evaluation mode
    all_true_tags = []
    all_pred_tags = []

    with torch.no_grad():  # Disable gradient calculation for evaluation
        for batch in dataloader:
            words, true_tags = batch

            # Get the model predictions
            outputs = model(words)
            predictions = torch.argmax(outputs, dim=-1)  # Get the tag with the highest score

            # Convert the predictions and true tags to lists
            true_tags = true_tags.view(-1).cpu().numpy()
            predictions = predictions.view(-1).cpu().numpy()

            # Mask out padding tokens (ignore_index for the '<PAD>' token)
            mask = (words.view(-1) != word2idx['<PAD>']).cpu().numpy()

            true_tags = true_tags[mask]
            predictions = predictions[mask]

            # Collect all true and predicted tags
            all_true_tags.extend(true_tags)
            all_pred_tags.extend(predictions)

    # Print classification report
    print(classification_report(all_true_tags, all_pred_tags, target_names=[key for key in tag2idx.keys()]))

evaluate_model(model, test_dataloader, word2idx, tag2idx)

              precision    recall  f1-score   support

           O       0.99      0.36      0.53     54074
           B       0.00      0.25      0.00       102
           I       0.00      0.32      0.01       192

    accuracy                           0.36     54368
   macro avg       0.33      0.31      0.18     54368
weighted avg       0.99      0.36      0.53     54368

